# dien data preprocess

In [1]:
import os
import tensorflow as tf
import numpy as np
import sys
import random
import time

## process_data.py

### 1.process meta

{'asin': '0001047566', 'salesRank': {'Books': 3628249}, 'imUrl': 'http://ecx.images-amazon.com/images/I/51FWARNTHCL.jpg', 'categories': [['Books']], 'title': 'Hand in Glove'}

{'asin': '0001053396', 
'description': 'Writing poetry has always been a way to express what is going on in the world around us. Unfortunately, sometimes what is around us is war. Acclaimed British actor Paul McGann reads poems that date from the eighteenth century to modern times in this moving audio anthology. McGann is originally from Liverpool, and he still retains a slight Northern accent. Its lilting cadence blends well with the rhythm and worldliness of these powerful poems. Although some of the pieces are well known, others are less so, and this makes the collection even more intriguing for lovers of literature.  R.A.P. &copy; AudioFile 2001, Portland, Maine--Copyright &#169; AudioFile, Portland, Maine', 
'title': 'War Poems: An Anthology of Poetry from the 18th Cantury to the Present Day', 'price': 17.99, 
'imUrl': 'http://ecx.images-amazon.com/images/I/51WTKK4V2DL.jpg', 
'salesRank': {'Books': 12249714}, 'categories': [['Books']]}

{'asin': '0000401048', 'title': "The rogue of publishers' row;: Confessions of a publisher (A Banner Book)", 'imUrl': 'http://ecx.images-amazon.com/images/I/41bchvIfgaL.jpg', 'related': {'also_viewed': ['068240103X']}, 'salesRank': {'Books': 6448843}, 'categories': [['Books']]}

{'asin': '0001019880', 'title': "Classic Soul Winner's New Testament Bible", 'price': 5.39, 'imUrl': 'http://ecx.images-amazon.com/images/I/61LcHUdvS1L.jpg', 'related': {'also_viewed': ['B003HMB5FC', '0834004593'], 'buy_after_viewing': ['031095360X', '0834004577']}, 'salesRank': {'Books': 9589258}, 'categories': [['Books']]}

{'asin': '0001048813', 'imUrl': 'http://ecx.images-amazon.com/images/I/41k5u0lr6NL.jpg', 'categories': [['Books']], 'title': 'Archer Christmas 4 Tape Pack'}

{'asin': '0001148427', 'salesRank': {'Books': 5806769}, 'imUrl': 'http://ecx.images-amazon.com/images/I/41tN4KuOXkL.jpg', 'categories': [['Books']], 'title': 'Sonatas - For Piano'}


In [3]:
meta_file = 'D:/workspace/data/dien/meta_books1000.json'

In [19]:
def process_meta(file):
    fi = open(file, "r")
    fo = open("item-info", "w")
    for line in fi:
        print(line)
        obj = eval(line)
        print('-'*20)
        print(obj)
        print('*'*20)
        cat = obj["categories"][0][-1]
        outline = obj["asin"] + "\t" + cat+ "\n"
        print(outline)
        fo.write(outline)

In [20]:
process_meta(meta_file)  # itemID cate_str

{'asin': '0001048791', 'salesRank': {'Books': 6334800}, 'imUrl': 'http://ecx.images-amazon.com/images/I/51MKP0T4DBL.jpg', 'categories': [['Books']], 'title': 'The Crucible: Performed by Stuart Pankin, Jerome Dempsey &amp; Cast'}

--------------------
{'asin': '0001048791', 'salesRank': {'Books': 6334800}, 'imUrl': 'http://ecx.images-amazon.com/images/I/51MKP0T4DBL.jpg', 'categories': [['Books']], 'title': 'The Crucible: Performed by Stuart Pankin, Jerome Dempsey &amp; Cast'}
********************
0001048791	Books

{'asin': '0001048775', 'description': 'William Shakespeare is widely regarded as the greatest playwright the world has seen. He produced an astonishing amount of work; 37 plays, 154 sonnets, and 5 poems. He died on 23rd April 1616, aged 52, and was buried in the Holy Trinity Church, Stratford.', 'title': 'Measure for Measure: Complete &amp; Unabridged', 'imUrl': 'http://ecx.images-amazon.com/images/I/5166EBHDQYL.jpg', 'salesRank': {'Books': 13243226}, 'categories': [['Books']]

### 2.process reviews

{"reviewerID": "ACNGUPJ3A3TM9", "asin": "0000013714", "reviewerName": "GCM", "helpful": [0, 0], "reviewText": "We use this type of hymnal at church.  I was looking for the same one; however, this wasn't it. It is a good hymnal, but there isn't enough information to find the version I need.", "overall": 4.0, "summary": "Nice Hymnal", "unixReviewTime": 1386028800, "reviewTime": "12 3, 2013"}

{"reviewerID": "A3BED5QFJWK88M", "asin": "0000013714", "reviewerName": "jimedd", "helpful": [0, 0], "reviewText": "Heavenly Highway HymnsI ordered this hymnal because I learned to read shaped note music when I was a teenager. I play piano but do not sing. I am 85 years old. This hymnal has mostof the songs I have learned over the years. It was exactly what I wanted and needed.It was in good condition and the price was right. I purchased this book from Amazon.", "overall": 4.0, "summary": "Heavenly Highway Hymns", "unixReviewTime": 1350345600, "reviewTime": "10 16, 2012"}

{"reviewerID": "A2SUAM1J3GNN3B", "asin": "0000013714", "reviewerName": "J. McDonald", "helpful": [2, 3], "reviewText": "I bought this for my husband who plays the piano.  He is having a wonderful time playing these old hymns.  The music  is at times hard to read because we think the book was published for singing from more than playing from.  Great purchase though!", "overall": 5.0, "summary": "Heavenly Highway Hymns", "unixReviewTime": 1252800000, "reviewTime": "09 13, 2009"}

{"reviewerID": "APOZ15IEYQRRR", "asin": "0000013714", "reviewerName": "maewest64", "helpful": [0, 0], "reviewText": "This is a large size hymn book which is great to be able to see the songs, notes, etc.  Quality was great, item was new!", "overall": 5.0, "summary": "Awesome Hymn Book", "unixReviewTime": 1362787200, "reviewTime": "03 9, 2013"}


In [13]:
reviews_file = 'D:/workspace/data/dien/reviews1000.json'

In [21]:
def process_reviews(file):
    fi = open(file, "r")
    user_map = {}
    fo = open("reviews-info", "w")
    for line in fi:
        obj = eval(line)
        userID = obj["reviewerID"]
        itemID = obj["asin"]
        rating = obj["overall"]
        time = obj["unixReviewTime"]
        outline = userID + "\t" + itemID + "\t" + str(rating) + "\t" + str(time)+"\n"
        print(outline)
        fo.write(outline)

In [22]:
process_reviews(reviews_file)  # userID itemID rating time

AH2L9G3DQHHAJ	0000000116	4.0	1019865600

A2IIIDRK3PRRZY	0000000116	1.0	1395619200

A1TADCM7YWPQ8M	0000000868	4.0	1031702400

AWGH7V0BDOJKB	0000013714	4.0	1383177600

A3UTQPQPM4TQO0	0000013714	5.0	1374883200

A8ZS0I5L5V31B	0000013714	5.0	1393632000

ACNGUPJ3A3TM9	0000013714	4.0	1386028800

A3BED5QFJWK88M	0000013714	4.0	1350345600

A2SUAM1J3GNN3B	0000013714	5.0	1252800000

APOZ15IEYQRRR	0000013714	5.0	1362787200

AYEDW3BFK53XK	0000013714	5.0	1325462400

A1KLCGLCXYP1U1	0000013714	3.0	1376092800

A37W6POFWIVG13	0000013714	5.0	1316131200

A2EIPZNHAEXZHJ	0000013714	4.0	1325030400

A1VAFVJFT58YI3	0000013714	5.0	1384300800

A9KTKY6BUR8U6	0000013714	1.0	1357516800

A2742OG8PK8KU6	0000013714	5.0	1358380800

A38AAPXSJN4C5G	0000015393	4.0	1239494400

A14A5Q8VJK5NLR	0000029831	4.0	1393286400

A3W2PX96K1BA3M	0000029831	5.0	1393200000

A2GKR2Q7MD8DG4	0000029831	5.0	1393286400

A1MC4E00RO5E9T	0000029831	5.0	1393200000

A23PISU0ZLW71C	0000029831	5.0	1393200000

A1KP12IVP6LAGL	0000038504	5.0	1232755200



A372BPDH0UAGNR	0001056107	5.0	952128000

A1AF30H2MPOO9	0001056530	4.0	966816000

A3QIVQ3SKET6TW	0001056530	4.0	964051200

AONMTGS00YTK7	0001057138	5.0	1314662400

ADDB0Y73L2CHU	0001057146	3.0	994118400

ADDB0Y73L2CHU	0001057170	3.0	991612800

A1CUCKRYAXFH1C	0001057324	5.0	1114300800

A2T5RBN49FVXZX	0001057324	4.0	1175040000

A2TAEHWKIREZP9	0001061100	5.0	1360281600

A1H01R0MEDEUC6	0001061100	5.0	1384819200

A373Y963BD0M2L	0001061127	5.0	1224806400

A142EFCHX4ZJ31	0001061127	5.0	1249430400

A25DDPF5TGEJT9	0001061127	5.0	1402099200

A1MV6SMIRQ5YU7	0001062395	5.0	1371340800

A12UFE6RPVTZJT	0001064487	4.0	1360281600

A3GFXEFR8FDX6P	0001064487	5.0	1309046400

A17K364R0ETIJJ	0001064487	5.0	1355961600

A1V9HZP9ONKV78	0001064487	5.0	1367280000

A53ZWHWGLPZQV	0001072986	5.0	1376611200

A3F1VVP8RKWFC	0001127748	5.0	1385856000

A1VBWYIOZQQIDQ	0001148427	5.0	1397779200

A15E0ZBE01DCDH	0001203010	5.0	1382745600

A2JS5MT00ZNGA2	0001203088	5.0	1357257600

A3ECAVKWPB26HV	0001360000	5.0	1196035200

A1

In [33]:
def manual_join(rev_file="reviews-info",meta_file="item-info",fout="jointed-new"):
    f_rev = open(rev_file, "r")
    user_map = {}
    item_list = []
    for line in f_rev:
        line = line.strip()
        items = line.split("\t")
        #loctime = time.localtime(float(items[-1]))
        #items[-1] = time.strftime('%Y-%m-%d', loctime)
        if items[0] not in user_map:
            user_map[items[0]]= []
        user_map[items[0]].append(("\t".join(items), float(items[-1])))  # {userID:[items,time_float]}
        item_list.append(items[1])  # [itemID]
        
    print('\n------user_map------\n',user_map)
    print('\n------item_list-----\n',item_list)
    
    f_meta = open(meta_file, "r")
    meta_map = {}
    for line in f_meta:
        arr = line.strip().split("\t")
        if arr[0] not in meta_map:
            meta_map[arr[0]] = arr[1]  # {itemID:cate_str}
            arr = line.strip().split("\t")
            
    print('\n========meta_map======\n',meta_map)
            
    fo = open(fout, "w")
    for key in user_map:
        sorted_user_bh = sorted(user_map[key], key=lambda x:x[1])
        print('key:{},sorted_user_bh:'.format(key),sorted_user_bh)
        for line, t in sorted_user_bh:  # items, time_float
            items = line.split("\t")
            asin = items[1]  # itemID
            j = 0
            while True:
                asin_neg_index = random.randint(0, len(item_list) - 1)
                asin_neg = item_list[asin_neg_index]  # 随机抽一个itemID作为负例，可以改成从meta_map.keys()里面抽取负例itemID，后边不报错
                if asin_neg == asin:
                    continue   # 与当前itemID重复则重新抽取负例itemID
                items[1] = asin_neg  # 将当前样本itemID替换为负例itemID且加上cate信息
                try:  # 我加的，因为数据不全，防止报错
                    outline1 = "0" + "\t" + "\t".join(items) + "\t" + meta_map[asin_neg]+"\n"
                except:
                    continue
                print('   outline1：',outline1)
                fo.write(outline1)  # 将替换过itemID的负样本的样本记录
                j += 1
                if j == 1:             #negative sampling frequency，j==1，则每个样本取一个负例
                    break
            if asin in meta_map:
                outline2 = "1" + "\t" + line + "\t" + meta_map[asin]+"\n"  # itemID有记录的正例，加上cate写入文件
                print('in outline2：',outline2)
                fo.write(outline2)
            else:
                outline2 = "1" + "\t" + line + "\t" + "default_cat"+"\n"  # 没有分类记录的itemID，加上默认cate写入文件
                print('out outline2：',outline2)
                fo.write(outline2)

In [34]:
manual_join(rev_file="reviews-info",meta_file="item-info",fout="jointed-new")  # label,userID,itemID,rating,time,cate


------user_map------
 {'AH2L9G3DQHHAJ': [('AH2L9G3DQHHAJ\t0000000116\t4.0\t1019865600', 1019865600.0)], 'A2IIIDRK3PRRZY': [('A2IIIDRK3PRRZY\t0000000116\t1.0\t1395619200', 1395619200.0)], 'A1TADCM7YWPQ8M': [('A1TADCM7YWPQ8M\t0000000868\t4.0\t1031702400', 1031702400.0)], 'AWGH7V0BDOJKB': [('AWGH7V0BDOJKB\t0000013714\t4.0\t1383177600', 1383177600.0)], 'A3UTQPQPM4TQO0': [('A3UTQPQPM4TQO0\t0000013714\t5.0\t1374883200', 1374883200.0)], 'A8ZS0I5L5V31B': [('A8ZS0I5L5V31B\t0000013714\t5.0\t1393632000', 1393632000.0)], 'ACNGUPJ3A3TM9': [('ACNGUPJ3A3TM9\t0000013714\t4.0\t1386028800', 1386028800.0)], 'A3BED5QFJWK88M': [('A3BED5QFJWK88M\t0000013714\t4.0\t1350345600', 1350345600.0)], 'A2SUAM1J3GNN3B': [('A2SUAM1J3GNN3B\t0000013714\t5.0\t1252800000', 1252800000.0)], 'APOZ15IEYQRRR': [('APOZ15IEYQRRR\t0000013714\t5.0\t1362787200', 1362787200.0)], 'AYEDW3BFK53XK': [('AYEDW3BFK53XK\t0000013714\t5.0\t1325462400', 1325462400.0)], 'A1KLCGLCXYP1U1': [('A1KLCGLCXYP1U1\t0000013714\t3.0\t1376092800', 13760928

in outline2： 1	A2ZZHMT58ZMVCZ	000100039X	5.0	1137456000	Books

key:A5K32C2QZN3WL,sorted_user_bh: [('A5K32C2QZN3WL\t000100039X\t4.0\t1100649600', 1100649600.0)]
   outline1： 0	A5K32C2QZN3WL	0000202010	4.0	1100649600	Books

in outline2： 1	A5K32C2QZN3WL	000100039X	4.0	1100649600	Books

key:A3BWLXLEOHMD01,sorted_user_bh: [('A3BWLXLEOHMD01\t000100039X\t5.0\t1246320000', 1246320000.0)]
   outline1： 0	A3BWLXLEOHMD01	000171287X	5.0	1246320000	Books

in outline2： 1	A3BWLXLEOHMD01	000100039X	5.0	1246320000	Books

key:A3S7ZMG2KGJ5BA,sorted_user_bh: [('A3S7ZMG2KGJ5BA\t000100039X\t5.0\t1369440000', 1369440000.0)]
   outline1： 0	A3S7ZMG2KGJ5BA	0000013714	5.0	1369440000	Books

in outline2： 1	A3S7ZMG2KGJ5BA	000100039X	5.0	1369440000	Books

key:AQK9X858KUY9U,sorted_user_bh: [('AQK9X858KUY9U\t000100039X\t4.0\t1267056000', 1267056000.0)]
   outline1： 0	AQK9X858KUY9U	0001018043	4.0	1267056000	Books

in outline2： 1	AQK9X858KUY9U	000100039X	4.0	1267056000	Books

key:A1VSHT4FFP768T,sorted_user_bh: [('A1VSHT4

in outline2： 1	A1UF72FGY08K6V	000102521X	5.0	1068508800	Books

key:A39J92OJUYHVNK,sorted_user_bh: [('A39J92OJUYHVNK\t000102521X\t4.0\t1033516800', 1033516800.0), ('A39J92OJUYHVNK\t0001857029\t5.0\t1052438400', 1052438400.0)]
   outline1： 0	A39J92OJUYHVNK	000100039X	4.0	1033516800	Books

in outline2： 1	A39J92OJUYHVNK	000102521X	4.0	1033516800	Books

   outline1： 0	A39J92OJUYHVNK	000100039X	5.0	1052438400	Books

out outline2： 1	A39J92OJUYHVNK	0001857029	5.0	1052438400	default_cat

key:A1NA603N9BJHLU,sorted_user_bh: [('A1NA603N9BJHLU\t0001026038\t5.0\t1296172800', 1296172800.0)]
   outline1： 0	A1NA603N9BJHLU	0001055178	5.0	1296172800	Books

out outline2： 1	A1NA603N9BJHLU	0001026038	5.0	1296172800	default_cat

key:A319KYEIAZ3SON,sorted_user_bh: [('A319KYEIAZ3SON\t0001035649\t5.0\t1125360000', 1125360000.0)]
   outline1： 0	A319KYEIAZ3SON	000100039X	5.0	1125360000	Books

in outline2： 1	A319KYEIAZ3SON	0001035649	5.0	1125360000	Books

key:ANIMV3SPDD8SH,sorted_user_bh: [('ANIMV3SPDD8SH\t0001042

key:A2ZS91Z7MVJR92,sorted_user_bh: [('A2ZS91Z7MVJR92\t000171287X\t5.0\t1368316800', 1368316800.0)]
   outline1： 0	A2ZS91Z7MVJR92	000100039X	5.0	1368316800	Books

in outline2： 1	A2ZS91Z7MVJR92	000171287X	5.0	1368316800	Books

key:A1PKXLGXSUTNQP,sorted_user_bh: [('A1PKXLGXSUTNQP\t000171287X\t5.0\t1391558400', 1391558400.0)]
   outline1： 0	A1PKXLGXSUTNQP	0000013714	5.0	1391558400	Books

in outline2： 1	A1PKXLGXSUTNQP	000171287X	5.0	1391558400	Books

key:A2YNQXG0ERQUD3,sorted_user_bh: [('A2YNQXG0ERQUD3\t000171287X\t5.0\t1385683200', 1385683200.0)]
   outline1： 0	A2YNQXG0ERQUD3	000100039X	5.0	1385683200	Books

in outline2： 1	A2YNQXG0ERQUD3	000171287X	5.0	1385683200	Books

key:A2WHGYP1X72H4E,sorted_user_bh: [('A2WHGYP1X72H4E\t000171287X\t5.0\t1354752000', 1354752000.0)]
   outline1： 0	A2WHGYP1X72H4E	000100039X	5.0	1354752000	Books

in outline2： 1	A2WHGYP1X72H4E	000171287X	5.0	1354752000	Books

key:A179R0UL62Q36Z,sorted_user_bh: [('A179R0UL62Q36Z\t000171287X\t5.0\t1345680000', 1345680000.0)]
 

In [39]:
def split_test(fin="jointed-new",fout="jointed-new-split-info"):
    fi = open(fin, "r")  # # label,userID,itemID,rating,time,cate
    fo = open(fout, "w")
    user_count = {}
    for line in fi:
        line = line.strip()
        user = line.split("\t")[1]
        if user not in user_count:
            user_count[user] = 0
        user_count[user] += 1  # 统计每个userID有几个样本
    fi.seek(0)
    i = 0  # 记录每个userID录入了几个样本
    last_user = "A26ZDKC53OP6JD"  # 上一个userID
    for line in fi:
        line = line.strip()
        user = line.split("\t")[1]
        if user == last_user:
            if i < user_count[user] - 2:  # 1 + negative samples，如果改变负样本抽样频率，这里也要改变
                outline = "20180118" + "\t" + line+"\n"
                print('user:{},outline:'.format(user),outline)
                fo.write(outline)
            else:
                outline = "20190119" + "\t" + line+"\n"
                print('user:{},outline:'.format(user),outline)
                fo.write(outline)
        else:
            last_user = user
            i = 0
            if i < user_count[user] - 2:
                outline = "20180118" + "\t" + line+"\n"
                print('user:{},outline:'.format(user),outline)
                fo.write(outline)
            else:
                outline = "20190119" + "\t" + line+"\n"
                print('user:{},outline:'.format(user),outline)
                fo.write(outline)
        i += 1

In [40]:
split_test()  # 真time，label，userID, itemID, rating, time, cate_str

user:AH2L9G3DQHHAJ,outline: 20190119	0	AH2L9G3DQHHAJ	000100039X	4.0	1019865600	Books

user:AH2L9G3DQHHAJ,outline: 20190119	1	AH2L9G3DQHHAJ	0000000116	4.0	1019865600	Books

user:A2IIIDRK3PRRZY,outline: 20190119	0	A2IIIDRK3PRRZY	0001360000	1.0	1395619200	Books

user:A2IIIDRK3PRRZY,outline: 20190119	1	A2IIIDRK3PRRZY	0000000116	1.0	1395619200	Books

user:A1TADCM7YWPQ8M,outline: 20190119	0	A1TADCM7YWPQ8M	000100039X	4.0	1031702400	Books

user:A1TADCM7YWPQ8M,outline: 20190119	1	A1TADCM7YWPQ8M	0000000868	4.0	1031702400	Books

user:AWGH7V0BDOJKB,outline: 20190119	0	AWGH7V0BDOJKB	000100039X	4.0	1383177600	Books

user:AWGH7V0BDOJKB,outline: 20190119	1	AWGH7V0BDOJKB	0000013714	4.0	1383177600	Books

user:A3UTQPQPM4TQO0,outline: 20190119	0	A3UTQPQPM4TQO0	000100039X	5.0	1374883200	Books

user:A3UTQPQPM4TQO0,outline: 20190119	1	A3UTQPQPM4TQO0	0000013714	5.0	1374883200	Books

user:A8ZS0I5L5V31B,outline: 20190119	0	A8ZS0I5L5V31B	000100039X	5.0	1393632000	Books

user:A8ZS0I5L5V31B,outline: 20190119	1	A8Z


user:A2Q4HZUGLXP6F1,outline: 20190119	1	A2Q4HZUGLXP6F1	000100039X	5.0	1381190400	Books

user:A3SGEA0N8OU613,outline: 20190119	0	A3SGEA0N8OU613	000171287X	4.0	1289347200	Books

user:A3SGEA0N8OU613,outline: 20190119	1	A3SGEA0N8OU613	000100039X	4.0	1289347200	Books

user:A1NGIUZJ9KFRV3,outline: 20190119	0	A1NGIUZJ9KFRV3	000171287X	5.0	1390953600	Books

user:A1NGIUZJ9KFRV3,outline: 20190119	1	A1NGIUZJ9KFRV3	000100039X	5.0	1390953600	Books

user:AUGZ7EUD67ROM,outline: 20190119	0	AUGZ7EUD67ROM	0001714538	5.0	1210032000	Books

user:AUGZ7EUD67ROM,outline: 20190119	1	AUGZ7EUD67ROM	000100039X	5.0	1210032000	Books

user:A3AKZ17Y3FK7QW,outline: 20190119	0	A3AKZ17Y3FK7QW	000171287X	3.0	1373932800	Books

user:A3AKZ17Y3FK7QW,outline: 20190119	1	A3AKZ17Y3FK7QW	000100039X	3.0	1373932800	Books

user:A2EPG54INKRW25,outline: 20190119	0	A2EPG54INKRW25	0001361155	5.0	1236384000	Books

user:A2EPG54INKRW25,outline: 20190119	1	A2EPG54INKRW25	000100039X	5.0	1236384000	Books

user:A1DW9QEARBGRFO,outline: 201901

## local_aggretor.py

In [42]:
fin = open("jointed-new-split-info", "r")  # # 真time，label，userID, itemID, rating, time, cate_str
ftrain = open("local_train", "w")
ftest = open("local_test", "w")

last_user = "0"
common_fea = ""


In [44]:
line_idx = 0
for line in fin:
    items = line.strip().split("\t")
    ds = items[0]  # 真time
    clk = int(items[1])  # label
    user = items[2]  # userid
    movie_id = items[3]  # itemid
    dt = items[5]  # time
    cat1 = items[6]  # cate_str

    if ds=="20180118":
        fo = ftrain
    else:
        fo = ftest
    if user != last_user:  # 每个用户单独记录
        movie_id_list = []
        cate1_list = []
        #print >> fo, items[1] + "\t" + user + "\t" + movie_id + "\t" + cat1 +"\t" + "" + "\t" + "" 
    else:
        history_clk_num = len(movie_id_list)
        cat_str = ""
        mid_str = ""
        for c1 in cate1_list:
            cat_str += c1 + ""
        for mid in movie_id_list:
            mid_str += mid + ""
        if len(cat_str) > 0: cat_str = cat_str[:-1]
        if len(mid_str) > 0: mid_str = mid_str[:-1]
        if history_clk_num >= 1:    # 8 is the average length of user behavior
            fo.write(items[1] + "\t" + user + "\t" + movie_id + "\t" + cat1 +"\t" + mid_str + "\t" + cat_str+"\n")
    last_user = user
    if clk:
        movie_id_list.append(movie_id)
        cate1_list.append(cat1)                
    line_idx += 1

## split_by_user.py

## generate_voc.py